Import the data that was cleaned via sql

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("clean_cookie_cats.csv")

df.head()
df.dtypes
df['ab_group'].value_counts()

ab_group
treatment    45489
control      44700
Name: count, dtype: int64

In [10]:
df.shape
df['ab_group'].value_counts()
df.dtypes

userid             int64
version           object
sum_gamerounds     int64
retention_1        int64
retention_7        int64
ab_group          object
dtype: object

Define the experiment formally 

In [11]:
control = df[df['ab_group'] == 'control']
treatment = df[df['ab_group'] == 'treatment']

just to see

In [12]:
summary = df.groupby('ab_group').agg(
    n_users=('userid', 'count'),
    retention1_rate=('retention_1', 'mean'),
    retention7_rate=('retention_7', 'mean'),
    avg_gamerounds=('sum_gamerounds', 'mean')
)

summary

,n_users,retention1_rate,retention7_rate,avg_gamerounds
ab_group,,,,
control,44700,0.448188,0.190201,52.456264
treatment,45489,0.442283,0.182000,51.298776


**Primary A/B test: 7-day retention**

1)Extract counts

In [13]:
n_control = control.shape[0]
n_treatment = treatment.shape[0]

success_control = control['retention_7'].sum()
success_treatment = treatment['retention_7'].sum()

p_control = success_control / n_control
p_treatment = success_treatment / n_treatment

p_control, p_treatment

(0.19020134228187918, 0.18200004396667327)

2)compute uplift

In [14]:
uplift = p_treatment - p_control
uplift

-0.008201298315205913

negative uplift means the treatement preformed worse

**Now we preform a Statistical significance test**

2 proportion z test

In [15]:
from statsmodels.stats.proportion import proportions_ztest

count = np.array([success_treatment, success_control])
nobs = np.array([n_treatment, n_control])

z_stat, p_value = proportions_ztest(count, nobs)

z_stat, p_value

(-3.164358912748191, 0.001554249975614329)

Interpretation rule:
	
    •	p < 0.05 → statistically significant difference
    
	•	p ≥ 0.05 → difference could be noise

**Confidence interval for the uplift **

i.e. “What range of uplift values is plausible?”

In [16]:
from statsmodels.stats.proportion import proportion_confint

ci_treat = proportion_confint(success_treatment, n_treatment, alpha=0.05, method='normal')
ci_ctrl  = proportion_confint(success_control, n_control, alpha=0.05, method='normal')

uplift_ci = (ci_treat[0] - ci_ctrl[0], ci_treat[1] - ci_ctrl[1])
uplift_ci

(-0.00810881578885217, -0.008293780841559656)

Secondary output 1-day retention

In [17]:
success_control_1 = control['retention_1'].sum()
success_treatment_1 = treatment['retention_1'].sum()

count_1 = np.array([success_treatment_1, success_control_1])
nobs_1 = np.array([n_treatment, n_control])

z1, p1 = proportions_ztest(count_1, nobs_1)

p1

0.07440965529691913

Engagement showed a negative trend but did not reach statistical significance.
bc Not statistically significant at α > 0.05

**Engagement metric: sum_gamerounds**

using t-test as data here is continous

In [18]:
from scipy.stats import ttest_ind

t_stat, p_gamerounds = ttest_ind(
    treatment['sum_gamerounds'],
    control['sum_gamerounds'],
    equal_var=False
)

p_gamerounds

0.37592438409326223

save results to ready for tableau

In [19]:
results = pd.DataFrame({
    'metric': ['retention_7', 'retention_1', 'avg_gamerounds'],
    'control': [
        p_control,
        success_control_1 / n_control,
        control['sum_gamerounds'].mean()
    ],
    'treatment': [
        p_treatment,
        success_treatment_1 / n_treatment,
        treatment['sum_gamerounds'].mean()
    ],
    'uplift': [
        p_treatment - p_control,
        (success_treatment_1 / n_treatment) - (success_control_1 / n_control),
        treatment['sum_gamerounds'].mean() - control['sum_gamerounds'].mean()
    ]
})

results

,metric,control,treatment,uplift
0,retention_7,0.190201,0.182000,-0.008201
1,retention_1,0.448188,0.442283,-0.005905
2,avg_gamerounds,52.456264,51.298776,-1.157488


In [21]:
results.to_csv("ab_test_results_summary.csv", index=False)